In [ ]:
import cv2
import os
import zipfile
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Paths
dataset_zip_path = '/content/drive/MyDrive/yolo_dataset_degraded'
extracted_path = '/content/yolo_dataset_degraded'
output_clahe_path = '/content/yolo_dataset_degraded_clahe'
output_zip_path = '/content/yolo_dataset_degraded_clahe.zip'

# Unzip the dataset
#!unzip "{dataset_zip_path}" -d "{extracted_path}"

# Function to apply CLAHE
def apply_clahe(input_folder, output_folder, clip_limit=2.0, tile_grid_size=(8,8)):
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(input_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img_clahe = clahe.apply(img)
            cv2.imwrite(os.path.join(output_folder, filename), img_clahe)

# Process train and val images
train_input = os.path.join(extracted_path, 'images', 'train')
train_output = os.path.join(output_clahe_path, 'images', 'train')
val_input = os.path.join(extracted_path, 'images', 'val')
val_output = os.path.join(output_clahe_path, 'images', 'val')

apply_clahe(train_input, train_output)
apply_clahe(val_input, val_output)

# Copy labels
!cp -r "{os.path.join(extracted_path, 'labels')}" "{os.path.join(output_clahe_path, 'labels')}"

# Zip the CLAHE dataset
def zipdir(path, ziph):
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(path, '..')))

with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir(output_clahe_path, zipf)

# Save to Drive
!cp "{output_zip_path}" "/content/drive/MyDrive/"

print(f"CLAHE dataset saved to: /content/drive/MyDrive/yolo_dataset_clahe.zip")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CLAHE dataset saved to: /content/drive/MyDrive/yolo_dataset_clahe.zip


In [ ]:
import cv2
import numpy as np
import os
import zipfile
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Paths
dataset_zip_path = '/content/drive/MyDrive/yolo_dataset_modified.zip'
extracted_path = '/content/yolo_dataset_modified'
output_degraded_path = '/content/yolo_dataset_degraded'
output_zip_path = '/content/yolo_dataset_degraded.zip'

# Clean up and unzip silently
!rm -rf "{extracted_path}"
!unzip -oq "{dataset_zip_path}" -d "{extracted_path}"

# Degradation function
def degrade_image(img):
    if len(img.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.astype(np.float32)
    img = (img * np.random.uniform(0.4, 0.7)) + np.random.uniform(-10, 10)
    img = np.clip(img, 0, 255).astype(np.uint8)
    if np.random.rand() > 0.5:
        img = cv2.GaussianBlur(img, (3, 3), 0)
    noise = np.random.randn(*img.shape) * np.random.uniform(5, 15)
    img = np.clip(img + noise, 0, 255).astype(np.uint8)
    return img

# Process folders
def degrade_dataset(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = cv2.imread(os.path.join(input_folder, filename))
            cv2.imwrite(os.path.join(output_folder, filename), degrade_image(img))

# Process train/val
degrade_dataset(os.path.join(extracted_path, 'images', 'train'),
                os.path.join(output_degraded_path, 'images', 'train'))
degrade_dataset(os.path.join(extracted_path, 'images', 'val'),
                os.path.join(output_degraded_path, 'images', 'val'))

# Copy labels
!cp -r "{os.path.join(extracted_path, 'labels')}" "{os.path.join(output_degraded_path, 'labels')}"

# Zip and save
with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(output_degraded_path):
        for file in files:
            zipf.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       output_degraded_path))
!cp "{output_zip_path}" "/content/drive/MyDrive/"

print("Degraded dataset saved to Drive as yolo_dataset_degraded.zip")

Mounted at /content/drive
Degraded dataset saved to Drive as yolo_dataset_degraded.zip


  adding: content/yolo_dataset_modified/ (stored 0%)
  adding: content/yolo_dataset_modified/images/ (stored 0%)
  adding: content/yolo_dataset_modified/images/train/ (stored 0%)
  adding: content/yolo_dataset_modified/images/train/2011_002558.jpg (deflated 1%)
  adding: content/yolo_dataset_modified/images/train/2008_006692.jpg (deflated 6%)
  adding: content/yolo_dataset_modified/images/train/2008_001402.jpg (deflated 0%)
  adding: content/yolo_dataset_modified/images/train/2010_004096.jpg (deflated 0%)
  adding: content/yolo_dataset_modified/images/train/2008_000399.jpg (deflated 6%)
  adding: content/yolo_dataset_modified/images/train/2009_002058.jpg (deflated 0%)
  adding: content/yolo_dataset_modified/images/train/2009_003781.jpg (deflated 0%)
  adding: content/yolo_dataset_modified/images/train/2008_003458.jpg (deflated 1%)
  adding: content/yolo_dataset_modified/images/train/2011_000585.jpg (deflated 6%)
  adding: content/yolo_dataset_modified/images/train/2008_004630.jpg (defl

In [ ]:
!pip install opencv-python

Processing training images...


100%|██████████| 13700/13700 [01:31<00:00, 149.49it/s]


Processing validation images...


100%|██████████| 3425/3425 [00:21<00:00, 158.14it/s]

Grayscale conversion complete!


Image shape: (500, 486, 3)
Unique values: [  5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241
 242 24

100%|██████████| 3425/3425 [00:17<00:00, 200.18it/s]

Conversion complete!


In [ ]:
!pip install ultralytics


Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profil

100%|██████████| 755k/755k [00:00<00:00, 13.9MB/s]

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1     37248  ultralytics.nn.modules.block.C2f             [192, 64, 1]                  
 16                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 

train: Scanning /content/yolo_dataset_real/labels/train... 13700 images, 0 backgrounds, 7998 corrupt: 100%|██████████| 13700/13700 [00:35<00:00, 390.94it/s]

...

                   all       1456       2462      0.534       0.45      0.468      0.286

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/2         0G      1.327      1.596       1.38          6        320: 100%|██████████| 1426/1426 [25:30<00:00,  1.07s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 182/182 [01:57<00:00,  1.55it/s]


                   all       1456       2462      0.579      0.515      0.522      0.336

2 epochs completed in 0.923 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 182/182 [01:39<00:00,  1.83it/s]


                   all       1456       2462       0.58      0.515      0.522      0.336
               bicycle       1301       2022      0.731      0.552      0.642      0.394
                   car        121        159      0.434      0.465       0.41       0.27
                  boat        164        281      0.576      0.527      0.515      0.344
Speed: 0.5ms preprocess, 59.3ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train
Quick test complete! Check 'runs/detect/train/' for results.


Streaminguitvoer ingekort tot de laatste 5000 regels.
Skipped 2010_004374.txt (no valid classes after filtering).
Skipped 2011_005382.txt (no valid classes after filtering).
Skipped 2010_003744.txt (no valid classes after filtering).
Skipped 2011_001210.txt (no valid classes after filtering).
Skipped 2012_002715.txt (no valid classes after filtering).
Skipped 2012_000320.txt (no valid classes after filtering).
Skipped 2009_001806.txt (no valid classes after filtering).
Skipped 2009_001598.txt (no valid classes after filtering).
Skipped 2011_000730.txt (no valid classes after filtering).
Skipped 2012_000583.txt (no valid classes after filtering).
Skipped 2012_000139.txt (no valid classes after filtering).
Skipped 2008_007610.txt (no valid classes after filtering).
Skipped 2010_006076.txt (no valid classes after filtering).
Skipped 2011_006004.txt (no valid classes after filtering).
Skipped 2010_006715.txt (no valid classes after filtering).
Skipped 2012_004298.txt (no valid classes afte

Dataset zipped at: /content/yolo_dataset_modified.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download started!
